# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [140]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [141]:
new_vacay=pd.read_csv("../output_data/weather.csv")
df=pd.DataFrame(new_vacay)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.93,125.41,57.72,100,52,1.45,TL,1597095866
1,Ushuaia,-54.80,-68.30,33.80,80,40,18.34,AR,1597095867
2,Torbay,47.67,-52.73,80.01,57,75,16.11,CA,1597095867
3,Asău,46.43,26.40,61.00,91,19,2.82,RO,1597095867
4,Kasongo-Lunda,-6.47,16.82,74.46,61,47,4.41,CD,1597095867
...,...,...,...,...,...,...,...,...,...
544,Terney,45.05,136.61,65.93,93,100,1.05,RU,1597095917
545,Ol'ga,43.74,135.29,67.71,96,100,4.79,RU,1597095917
546,Prince Rupert,54.32,-130.32,57.20,71,90,2.24,CA,1597095886
547,High Level,58.52,-117.14,64.40,63,75,5.82,CA,1597095917


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [142]:
gmaps.configure(api_key=g_key)

#Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

#Store Humidity in humidity
humidity = df["Humidity"]


In [143]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=5)

#Add layer
fig.add_layer(heat_layer)

#Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [144]:
cities_weather=df.loc[(df["Max Temp"]>70) & (df["Max Temp"]<80)
                      & (df["Wind Speed"]<10) & (df["Cloudiness"]==0)]
new_weather_condition= cities_weather.dropna(how="any")
new_weather_condition

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Ilhabela,-23.78,-45.36,70.47,68,0,4.72,BR,1597095867
294,Marsá Maţrūḩ,31.35,27.25,77.38,82,0,7.61,EG,1597095897
298,Yalova,40.66,29.28,71.60,88,0,5.82,TR,1597095897
396,Kaka,37.35,59.61,77.56,26,0,1.43,TM,1597095906
411,Binzhou,37.37,118.02,76.50,83,0,3.51,CN,1597095907
447,Morros,-9.45,-46.30,79.16,35,0,5.73,BR,1597095910
492,Cerveteri,41.99,12.09,77.00,88,0,4.70,IT,1597095913
493,São Miguel do Araguaia,-13.28,-50.16,78.80,32,0,3.69,BR,1597095913
501,Yinzhu,35.88,119.98,77.00,94,0,2.24,CN,1597095914
514,Viranşehir,37.24,39.76,73.99,68,0,3.24,TR,1597095915


In [145]:
hotel_df=new_weather_condition.loc[:,["City","Country","Lat","Lng"]]
hotel_df

,City,Country,Lat,Lng
8,Ilhabela,BR,-23.78,-45.36
294,Marsá Maţrūḩ,EG,31.35,27.25
298,Yalova,TR,40.66,29.28
396,Kaka,TM,37.35,59.61
411,Binzhou,CN,37.37,118.02
447,Morros,BR,-9.45,-46.30
492,Cerveteri,IT,41.99,12.09
493,São Miguel do Araguaia,BR,-13.28,-50.16
501,Yinzhu,CN,35.88,119.98
514,Viranşehir,TR,37.24,39.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [146]:
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
8,Ilhabela,BR,-23.78,-45.36,
294,Marsá Maţrūḩ,EG,31.35,27.25,
298,Yalova,TR,40.66,29.28,
396,Kaka,TM,37.35,59.61,
411,Binzhou,CN,37.37,118.02,
447,Morros,BR,-9.45,-46.30,
492,Cerveteri,IT,41.99,12.09,
493,São Miguel do Araguaia,BR,-13.28,-50.16,
501,Yinzhu,CN,35.88,119.98,
514,Viranşehir,TR,37.24,39.76,


In [152]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    # getting city name, latitude, longitude from dataframe
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # adding keyword to params dictionary
    params["location"] = f"{lat},{lng}"
    
    # assembling the url and making API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    
    response_json = requests.get(base_url, params=params).json()
    #real_response=json.dumps(response_json, indent=4,sort_keys=True)
   # print(real_response)
    
   
    output = response_json['results']
    # saving the hotel name to dataframe
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/skipping.")
        
    print("--------------------------")







Retrieving Results for Index 8: Ilhabela.
Closest hotel in Ilhabela is Viranşehir.
--------------------------
Retrieving Results for Index 294: Marsá Maţrūḩ.
Closest hotel in Marsá Maţrūḩ is Viranşehir.
--------------------------
Retrieving Results for Index 298: Yalova.
Closest hotel in Yalova is Viranşehir.
--------------------------
Retrieving Results for Index 396: Kaka.
Closest hotel in Kaka is Viranşehir.
--------------------------
Retrieving Results for Index 411: Binzhou.
Closest hotel in Binzhou is Viranşehir.
--------------------------
Retrieving Results for Index 447: Morros.
Closest hotel in Morros is Viranşehir.
--------------------------
Retrieving Results for Index 492: Cerveteri.
Closest hotel in Cerveteri is Viranşehir.
--------------------------
Retrieving Results for Index 493: São Miguel do Araguaia.
Closest hotel in São Miguel do Araguaia is Viranşehir.
--------------------------
Retrieving Results for Index 501: Yinzhu.
Closest hotel in Yinzhu is Viranşehir.
-----

In [153]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [154]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))